## Research Notebook for 01 XAU Bot

This notebook looks into a simple SMAcrossover strategy using backtesting.py libray and mt5 as a data feed. the following are the steps that are applied:

1. get data from mt5 using settings of desired asset and timeframe 
2. using class from backtesting.py, backtest the strategy 
3. optimize the strategy for the best parameters 
4. visualize using graphical methods. 






In [1]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import cycler
%matplotlib inline

In [3]:
def plt_dark():
       colors = cycler('color',
                     ['#669FEE', '#66EE91', '#9988DD',
                     '#EECC55', '#88BB44', '#FFBBBB'])
       plt.rc('figure', facecolor='#313233')
       plt.rc('axes', facecolor="#313233", edgecolor='none',
              axisbelow=True, grid=True, prop_cycle=colors,
              labelcolor='gray')
       plt.rc('grid', color='474A4A', linestyle='solid')
       plt.rc('xtick', color='gray')
       plt.rc('ytick', direction='out', color='gray')
       plt.rc('legend', facecolor="#313233", edgecolor="#313233")
       plt.rc("text", color="#C9C9C9")
       plt.rc('figure', facecolor='#313233')
plt_dark()

In [10]:

# Settings 
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_H1
start_date = datetime(2022, 8, 1)
end_date = datetime.now()

# Function 
def GetMTData():
    mt5.initialize()
    bars = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
    bars['time'] = pd.to_datetime(bars['time'], unit='s')
    bars.columns = bars.columns.str.capitalize()
    return bars

df  = GetMTData()
# xauusd.tail()

class SMAcross(Strategy):
    
    n1 = 20
    n2 = 50
    
    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)
        
        
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
        

bt = Backtest(df, SMAcross, cash=100000, commission= 0.002, exclusive_orders=True)
            
output = bt.run()

output



    
    

C:\Users\User\AppData\Local\Temp\ipykernel_33300\4193372537.py:36: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SMAcross, cash=100000, commission= 0.002, exclusive_orders=True)


Start                                     0.0
End                                    9631.0
Duration                               9631.0
Exposure Time [%]                   99.449751
Equity Final [$]                  70479.74938
Equity Peak [$]                  100443.64544
Return [%]                         -29.520251
Buy & Hold Return [%]                22.21466
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -33.129117
Avg. Drawdown [%]                    -8.47181
Max. Drawdown Duration                 9569.0
Avg. Drawdown Duration                2394.75
# Trades                                205.0
Win Rate [%]                        30.243902
Best Trade [%]                       5.916861
Worst Trade [%]                     -2.228087
Avg. Trade [%]                    

In [9]:
bt.plot()


Row(id='3131', ...)

In [6]:
optim = bt.optimize(n1 = range(20, 130, 10),
                    n2 = range (20, 130, 10),
                    constraint = lambda x: x.n2 - x.n1 >= 5,
                    maximize = 'Return [%]')
bt.plot()

  0%|          | 0/10 [00:00<?, ?it/s]

Row(id='2251', ...)

In [7]:
optim

Start                                     0.0
End                                    9631.0
Duration                               9631.0
Exposure Time [%]                   97.425249
Equity Final [$]                 113013.51202
Equity Peak [$]                   119813.8555
Return [%]                          13.013512
Buy & Hold Return [%]                22.21466
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.992364
Avg. Drawdown [%]                   -0.996624
Max. Drawdown Duration                 4692.0
Avg. Drawdown Duration             112.409639
# Trades                                 83.0
Win Rate [%]                        38.554217
Best Trade [%]                       7.051798
Worst Trade [%]                     -4.856016
Avg. Trade [%]                    

# Martingale Risk management


In [10]:
# Settings
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_H1
start_date = datetime(2020, 1, 1)
end_date = datetime.now()

# Function to fetch data from MetaTrader
def get_mt_data():
    mt5.initialize()
    bars = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
    bars['time'] = pd.to_datetime(bars['time'], unit='s')
    bars.columns = bars.columns.str.capitalize()
    return bars

# Fetch historical price data
df = get_mt_data()

# Strategy class with Martingale risk management
class SMAcrossMartingale(Strategy):
    n1 = 20
    n2 = 50
    initial_position_size = 10.0  # Initial position size
    martingale_multiplier = 1.5  # Multiplier for position size after a loss
    current_position_size = initial_position_size

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy(size=self.current_position_size)  # Buy with the current position size
        elif crossover(self.sma2, self.sma1):
            self.sell(size=self.current_position_size)  # Sell with the current position size

    def on_loss(self, trade):
        # Update position size using Martingale strategy
        self.current_position_size *= self.martingale_multiplier

    def on_win(self, trade):
        # Reset position size to initial value after a winning trade
        self.current_position_size = self.initial_position_size

# Backtest with Martingale strategy
bt = Backtest(df, SMAcrossMartingale, cash=100000, commission=0.002, exclusive_orders=True)
output = bt.run()

# Print backtest results
output

C:\Users\User\AppData\Local\Temp\ipykernel_14984\1735302736.py:46: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SMAcrossMartingale, cash=100000, commission=0.002, exclusive_orders=True)


Start                                     0.0
End                                    3714.0
Duration                               3714.0
Exposure Time [%]                   97.604307
Equity Final [$]                  100524.0932
Equity Peak [$]                   101732.7088
Return [%]                           0.524093
Buy & Hold Return [%]                9.681916
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.154782
Avg. Drawdown [%]                   -0.135003
Max. Drawdown Duration                 2046.0
Avg. Drawdown Duration              81.272727
# Trades                                 76.0
Win Rate [%]                        36.842105
Best Trade [%]                       5.916861
Worst Trade [%]                      -1.68687
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='2251', ...)

In [11]:
optim = bt.optimize(n1 = range(20, 130, 10),
                    n2 = range (20, 130, 10),
                    constraint = lambda x: x.n2 - x.n1 >= 5,
                    maximize = 'Return [%]')
optim

  0%|          | 0/10 [00:00<?, ?it/s]

Start                                     0.0
End                                    3714.0
Duration                               3714.0
Exposure Time [%]                    89.74428
Equity Final [$]                  103162.1752
Equity Peak [$]                   103925.1126
Return [%]                           3.162175
Buy & Hold Return [%]                9.681916
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.365549
Avg. Drawdown [%]                   -0.118491
Max. Drawdown Duration                 1673.0
Avg. Drawdown Duration              52.709677
# Trades                                 20.0
Win Rate [%]                             45.0
Best Trade [%]                       7.385388
Worst Trade [%]                     -2.224525
Avg. Trade [%]                    